In [1]:
import scipy.io as sio
import torch
from torch. utils.data import TensorDataset, DataLoader
import numpy as np
import h5py

In [2]:
import sys
import os
# Get the parent directory of the current folder
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the parent directory to sys.path so Python can find the helper folder
sys.path.append(parent_dir)

# to load models
from train.Pix2Pix.helperPix2Pix.discriminator_model import Discriminator
from train.Pix2Pix.helperPix2Pix.generator_model import Generator, Generator_FineTune
from helper.utils import CNN_Est2, CNN_Est
from helper.utils_transfer import FineTuneModel2

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [4]:
domain = "mapBased"
for snr in np.arange(-25, 35, 5):  
    print(f"snr {snr}/30")

snr -25/30
snr -20/30
snr -15/30
snr -10/30
snr -5/30
snr 0/30
snr 5/30
snr 10/30
snr 15/30
snr 20/30
snr 25/30
snr 30/30


In [5]:
# Load the .mat file
if domain == "DeepMIMO":
    row = '3500_3516'
    dataFile = '../../DeepMIMOv2/DeepMIMO_Data/Static_BS16/freq_symb_1ant_612sub_ver4/Gan_' + str(snr) +'_dBOutdoor1_3p4_1ant_612subcs_Row_' + str(row) + '.mat'
    mat_data = h5py.File(dataFile, 'r')     # load Y, H, N
    
elif domain == "mapBased":
    dataFile = '../../CDL Customization/Data/ver39_/' + str(snr) + 'dB/1_mapBaseData.mat'
    mat_data = h5py.File(dataFile, 'r')     # load Y, H, N
    
H_data = np.array(mat_data['H_data'])                # Shape: (N, 2, 612, 14 )
H_linear_data = np.array(mat_data['H_linear_data'])  
    # H_equalized_data = np.array(mat_data['H_equalized_data'])  
Y_data = np.array(mat_data['Y_data'])                
Noise = np.array(mat_data['Noise'])                  

mat_data2 = sio.loadmat('../../DeepMIMOv2/DeepMIMO_Data/TransmitedGrid.mat') # load X (transmitted grid)
X = np.transpose(mat_data2['X'], (3, 2, 1, 0)) 

/home/thien/miniconda3/envs/Torch_GPU/lib/python3.8/site-packages/scipy/io/matlab/_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [6]:
# Verify that all tensors have the same first dimension
num_samples = H_data.shape[0]

# Randomly select {num_samples} unique indices from 10000 samples of X
random_indices = np.random.choice(X.shape[0], num_samples, replace=False)
# Extract the selected samples
X_data = X[random_indices, :, :, :]  # Shape: (14, 612, 2, {num_samples})

In [7]:
# Convert to PyTorch tensors
# Assuming complex data; split into real and imaginary if needed
H_data = torch.tensor(H_data, dtype=torch.float32).to(device)
# H_linear_data = torch.tensor(H_linear_data, dtype=torch.float32).to(device)
# H_equalized_data = torch.tensor(H_equalized_data, dtype=torch.float32).to(device)
Y_data = torch.tensor(Y_data, dtype=torch.float32).to(device)
Noise = torch.tensor(Noise, dtype=torch.float32).to(device)
X_data = torch.tensor(X_data, dtype=torch.float32).to(device)

In [8]:
# Loading trained-only models
model_path = '../model/static/CNN/BS16/3500_3516/ver27_/'
LS_CNN_trained = CNN_Est2().to(device)
checkpoint = (torch.load(model_path + str(snr)+'dB/CNN_1_LS_CNN_model.pth'))
LS_CNN_trained.load_state_dict(checkpoint["model_state_dict"])
LS_CNN_trained.eval()
#--------------------------------------------------
LI_CNN_trained = CNN_Est2().to(device)
checkpoint = (torch.load(model_path + str(snr)+'dB/CNN_1_LS_LI_CNN_model.pth'))
LS_CNN_trained.load_state_dict(checkpoint["model_state_dict"])
LI_CNN_trained.eval()
#--------------------------------------------------
epoc_load = 40 # epoch to load saved models
model_path = '../model/static/GAN/BS16/3500_3516/ver17_/'
LS_GAN_trained = Generator().to(device)
checkpoint = (torch.load(model_path + str(snr) + 'dB/' +str(epoc_load) + 'epoc_G_1_LS_GAN_model.pth'))
LS_GAN_trained.load_state_dict(checkpoint["model_state_dict"])
LS_GAN_trained.eval()
#--------------------------------------------------
LI_GAN_trained = Generator().to(device) 
checkpoint = (torch.load(model_path + str(snr) + 'dB/' +str(epoc_load) + 'epoc_G_1_LS_LI_GAN_model.pth'))
LS_GAN_trained.load_state_dict(checkpoint["model_state_dict"])
LI_GAN_trained.eval()

############################################################
# Loading transferred models
model_path = '../transfer/transferd_model/static/'
temp_model = CNN_Est()
LS_CNN_transfer = FineTuneModel2(temp_model).to(device)
if snr < 0:
    checkpoint = (torch.load(model_path + 'CNN/ver11_/' + str(snr)+'dB/LS_CNN_model.pth'))
else:
    checkpoint = (torch.load(model_path + 'CNN/ver10_/' + str(snr)+'dB/LS_CNN_model.pth'))
LS_CNN_transfer.load_state_dict(checkpoint["model_state_dict"])
LS_CNN_transfer.eval()
#--------------------------------------------------
LI_CNN_transfer = FineTuneModel2(temp_model).to(device)
if snr < 0:
    checkpoint = (torch.load(model_path + 'CNN/ver11_/' + str(snr)+'dB/LS_LI_CNN_model.pth'))
else:
    checkpoint = (torch.load(model_path + 'CNN/ver10_/' + str(snr)+'dB/LS_LI_CNN_model.pth'))
LI_CNN_transfer.load_state_dict(checkpoint["model_state_dict"])
LI_CNN_transfer.eval()
#--------------------------------------------------
model_path = '../transfer/transferd_model/static/'
temp_model = Generator()
LS_GAN_transfer = Generator_FineTune(temp_model).to(device)
checkpoint = (torch.load(model_path + 'GAN/ver9_/' + str(snr)+'dB/LS_GAN_G_model.pth'))
LS_GAN_transfer.load_state_dict(checkpoint["model_state_dict"])
LS_GAN_transfer.eval()
#--------------------------------------------------
LI_GAN_transfer = Generator_FineTune(temp_model).to(device)
checkpoint = (torch.load(model_path + 'GAN/ver9_/' + str(snr)+'dB/LS_LI_GAN_G_model.pth'))
LI_GAN_transfer.load_state_dict(checkpoint["model_state_dict"])
LI_GAN_transfer.eval()

/tmp/ipykernel_3105030/2808721120.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = (torch.load(model_path + str(snr)+'dB/CNN_1_LS_CNN_model.pth'))
/tmp/ipyke

Generator_FineTune(
  (initial_down): Conv2d(1, 64, kernel_size=(8, 4), stride=(2, 1), padding=(1, 1), padding_mode=reflect)
  (act_tanh): Tanh()
  (act_leaky): LeakyReLU(negative_slope=0.01)
  (act_relu): ReLU()
  (down1): Block(
    (batch_norm): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv): Conv2d(64, 128, kernel_size=(8, 3), stride=(2, 1), padding=(1, 1), bias=False)
    (activation): LeakyReLU(negative_slope=0.01)
  )
  (down2): Block(
    (batch_norm): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv): Conv2d(128, 256, kernel_size=(8, 3), stride=(2, 1), padding=(1, 0), bias=False)
    (activation): LeakyReLU(negative_slope=0.01)
  )
  (down3): Block(
    (batch_norm): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv): Conv2d(256, 512, kernel_size=(7, 3), stride=(2, 1), bias=False)
    (activation): LeakyReLU(negative_slope=0.01)
  )
  (down4): Block(


In [9]:
N = 2
k = 3
d = 2

L = torch.arange(N * k * d * d).view(N, k, d, d)
index = torch.tensor([0,1,0,0]).view(N,-1)
L[torch.arange(L.size(0)).unsqueeze(1), index]

tensor([[[[ 0,  1],
          [ 2,  3]],

         [[ 4,  5],
          [ 6,  7]]],


        [[[12, 13],
          [14, 15]],

         [[12, 13],
          [14, 15]]]])

In [10]:
pilot = sio.loadmat('../../DeepMIMOv2/DeepMIMO_Data/pilot_value.mat')
pilot_Indices = pilot['pilots_idx']
pilot_Symbols = pilot['pilot_tx']

pilot_Indices = torch.tensor(pilot_Indices, dtype=torch.int64).to(device)
pilot_Symbols = torch.tensor(pilot_Symbols, dtype=torch.complex64).to(device)

# Continue Coding from here

In [ ]:
from helper_BER import helperLinearInterp, linear_extrapolation_matrix
from helper.utils import minmaxScaler
# Create TensorDataset
dataset = TensorDataset(H_data, X_data, Noise)

# Create DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


# Test the dataloader
for batch_idx, batch in enumerate(dataloader):
    h, x, noise = batch
        # h == tensor [32,2,612,14] , x == [32,2,612,14], noise == [32,2,612,14]
    # creating complex values
    h_complex = torch.complex(h[:,0,:,:], h[:,1,:,:])
    x_complex = torch.complex(x[:,0,:,:], x[:,1,:,:])
    n_complex = torch.complex(noise[:,0,:,:], noise[:,1,:,:])
    Y_noise = h_complex * x_complex + n_complex # Element-wise multiplication # 32, 612, 14
    print(f"Batch shapes: H: {h.shape}, X: {x.shape}, Noise: {noise.shape}")
    H_equalized, H_linear =  helperLinearInterp(Y_noise, pilot_Indices, pilot_Symbols)
    
    H_LS  = torch.cat((torch.tensor(H_equalized.real).unsqueeze(1), torch.tensor(H_equalized.imag).unsqueeze(1)), dim=1)  # Shape: [32, 2, 612, 14]
    H_LI  = torch.cat((torch.tensor(H_linear.real).unsqueeze(1), torch.tensor(H_linear.imag).unsqueeze(1)), dim=1)  # Shape: [32, 2, 612, 14]

    # normalized data
    H_LS_normd, test_LS_min, test_LS_max = minmaxScaler(H_LS)
    H_LI_normd, test_LI_min, test_LI_max = minmaxScaler(H_LI)
        # testData_LS_normd = [32, 2, 612, 14] tensor, cpu 
        # test_LS_min = [32, 2] tensor, device  -- min of [32, real-imag]
    
    min_maxLS = torch.stack((test_LS_min, test_LS_max), dim=1)
    min_maxLI = torch.stack((test_LI_min, test_LI_max), dim=1)
        # [32, 2, 2] -- [:, 0, 0] = min, real       [:, 0, 1] = min, imag 
        #               [:, 1, 0] = max, real       [:, 1, 1] = max, imag
        
        
    
    break
    


Batch shapes: H: torch.Size([32, 2, 612, 14]), X: torch.Size([32, 2, 612, 14]), Noise: torch.Size([32, 2, 612, 14])


/home/thien/Hprediction/one_shot_Hest_cleanver/Torch_code/BER_calulation/helper_BER.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  H_linear = torch.tensor(H_linear, dtype=torch.complex64).to(H_equalized.device)
